# Functions

This page specifically focuses on examining functions in SQL that are exclusively designed for use with window functions. 

---

The following cell sets up a Docker container with SQL that we'll use as an example. 

In [9]:
docker run -d --rm\
    --name window_functions_example\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5

docker exec -i window_functions_example psql -U postgres -d postgres << EOF
CREATE TABLE example_table (
    some_value INT NOT NULL,
    "group" TEXT NOT NULL,
    group2 TEXT NOT NULL
);
INSERT INTO example_table (some_value, "group", group2) VALUES
(3, 'C', 'X'),
(3, 'C', 'Y'),
(1, 'C', 'X'),
(4, 'B', 'Y'),
(3, 'A', 'X'),
(1, 'D', 'Y'),
(2, 'A', 'Z'),
(4, 'A', 'X'),
(2, 'A', 'X'),
(0, 'D', 'X'),
(1, 'D', 'Y'),
(4, 'A', 'Y'),
(4, 'B', 'Z'),
(0, 'D', 'X'),
(0, 'D', 'Z'),
(0, 'D', 'X'),
(2, 'D', 'Y'),
(4, 'C', 'Z'),
(4, 'A', 'X'),
(2, 'B', 'X'),
(3, 'A', 'Z'),
(1, 'C', 'Y'),
(3, 'D', 'X'),
(2, 'A', 'X'),
(4, 'B', 'X');

SELECT * FROM example_table LIMIT 5;
EOF

CREATE TABLE
INSERT 0 25
 some_value | group | group2 
------------+-------+--------
          3 | C     | X
          3 | C     | Y
          1 | C     | X
          4 | B     | Y
          3 | A     | X
(5 rows)



**Note** don't forget to stop container after all.

In [14]:
docker stop window_functions_example

window_functions_example


## Values shifting (LAG/LEAD)

You can shift values over the window by using `LAG` functions for shifting down and `LEAD` funtions for shifting up.

---

The following cell demonstrates the application of the `LAG` and `LEAD` functions to our example.

In [10]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    "group",
    LAG("group") OVER () lag_step_1,
    LEAD("group") OVER () lead_step_1,
    LAG("group", 3) OVER () lag_step3,
    LEAD("group", 3) OVER () lag_step3
FROM example_table;
EOF

 group | lag_step_1 | lead_step_1 | lag_step3 | lag_step3 
-------+------------+-------------+-----------+-----------
 C     |            | C           |           | B
 C     | C          | C           |           | A
 C     | C          | B           |           | D
 B     | C          | A           | C         | A
 A     | B          | D           | C         | A
 D     | A          | A           | C         | A
 A     | D          | A           | B         | D
 A     | A          | A           | A         | D
 A     | A          | D           | D         | A
 D     | A          | D           | A         | B
 D     | D          | A           | A         | D
 A     | D          | B           | A         | D
 B     | A          | D           | D         | D
 D     | B          | D           | D         | D
 D     | D          | D           | A         | C
 D     | D          | D           | B         | A
 D     | D          | C           | D         | B
 C     | D          | A         

The result of applying `LAG` and `LEAD` produces new variables that can be represented as:

- For `LAG`:  $x'_i = x_{i-s}$ 
- For `LEAD`: $x'_i=x_{i+s}$

Where:

-  $i$ position of the row during generaiton.
-  $x_i$ is the original group value at the $i$-th position.
-  $s$ is the step value specified as the second argument of the function. 

### Partition by

If you include a `PARTITION BY` clause with the `LAG` window function, it will define groups of values within which the shifting operation will take place.

---

This example demonstrates shifting the `group` values within groups defined by `group2`. 

In [13]:
docker exec -i window_functions_example psql -U postgres -d postgres << EOF
SELECT 
    "group", group2,
    LAG("group") OVER (PARTITION BY group2)
FROM example_table;
EOF

 group | group2 | lag 
-------+--------+-----
 C     | X      | 
 C     | X      | C
 A     | X      | C
 A     | X      | A
 A     | X      | A
 D     | X      | A
 D     | X      | D
 D     | X      | D
 A     | X      | D
 B     | X      | A
 D     | X      | B
 A     | X      | D
 B     | X      | A
 C     | Y      | 
 D     | Y      | C
 C     | Y      | D
 D     | Y      | C
 D     | Y      | D
 A     | Y      | D
 B     | Y      | A
 A     | Z      | 
 C     | Z      | A
 D     | Z      | C
 B     | Z      | D
 A     | Z      | B
(25 rows)



The result demonstrates that the values were shifted only within their respective groups (windows) defined by `group2`.